# Testing EasyOCR To Extract Text From Image

In [69]:
import easyocr
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Tuple

In [4]:
image_path = './exampleFlights.png'

## Extract Text from Image

In [70]:
def extract_text_and_positions(image_path: str) -> Tuple[list, list]:
    reader = easyocr.Reader(['en'])
    image = cv2.imread(image_path)
    results = reader.readtext(image, paragraph=True, min_size=0)

    detect_boxes = reader.detect(image, min_size=0)

    return results, detect_boxes


In [ ]:
ocr_results, detect_boxes = extract_text_and_positions(image_path)

In [ ]:
len(detect_boxes[0][0])

In [ ]:
len(ocr_results)

In [ ]:
ocr_results[:2]

## Visualise detected bounding box from image

In [77]:
def visualise_bounding_box_in_image(image_path: str, results: list):

    image = cv2.imread(image_path)
    # for (bbox, text, prob) in results:
    for (bbox, text) in results:
        # Extract the bounding box coordinates
        (top_left, top_right, bottom_right, bottom_left) = bbox
        top_left = tuple([int(coord) for coord in top_left])
        bottom_right = tuple([int(coord) for coord in bottom_right])

        # Draw the rectangle around the detected text
        cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)

        # Optionally, display the detected text on the image
        cv2.putText(image, text, (top_left[0], top_left[1] - 5), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)

    # Convert image from BGR (OpenCV format) to RGB (matplotlib format)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image with bounding boxes
    plt.figure(figsize=(10, 10))
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()

In [ ]:
visualise_bounding_box_in_image(image_path, ocr_results)

In [ ]:
ocr_results

In [84]:
def analyze_and_structure_data(ocr_results):
    # Sort results by vertical position (y-coordinate)
    sorted_results = sorted(ocr_results, key=lambda x: x[0][0][1])

    # Group text by rows based on y-coordinate proximity
    rows = []
    current_row = []
    last_y = None
    y_threshold = 10  # Adjust this value based on your image

    for result in sorted_results:
        # box, text, conf = result
        box, text = result
        current_y = box[0][1]

        if last_y is None or abs(current_y - last_y) <= y_threshold:
            # current_row.append((box, text, conf))
            current_row.append((box, text))
        else:
            rows.append(current_row)
            # current_row = [(box, text, conf)]
            current_row = [(box, text)]
        last_y = current_y

    if current_row:
        rows.append(current_row)

    # Sort each row by horizontal position (x-coordinate)
    for row in rows:
        row.sort(key=lambda x: x[0][0][0])

    return rows

In [86]:
rows = analyze_and_structure_data(ocr_results)

In [90]:
def create_dataframe(structured_data):
    # Determine the maximum number of columns
    max_columns = max(len(row) for row in structured_data)

    # Create a list of dictionaries for the DataFrame
    data = []
    for row in structured_data:
        row_data = {}
        # for i, (_, text, _) in enumerate(row):
        for i, (_, text) in enumerate(row):
            row_data[f'Column_{i+1}'] = text
        data.append(row_data)

    # Create DataFrame
    df = pd.DataFrame(data)
    return df

In [ ]:
create_dataframe(rows)